### Basis of this idea

Use a simple branching algorithm to focus on a subset of warehouses to open. Only solve the full MIP when necessary.

This will give me some practice in working with Julia's built-in free solvers.

In [81]:
# load in base optimisation libraries from Julia
using Dates
using Printf
using JuMP
using Cbc
using Clp

function read_input(input_data)
    """
    Read in the input data from string to 
    numeric values.
    
    Parameters
    ----------
    input_data: string input data.
    
    Returns
    -------
    N: number of warehouses
    
    M: number of customers
    
    warehouses: array of setup cost (s_i), capacity (cap_i),
    and location co-ordinates (x_i, y_i) for each warehouse.
    
    customers: array of demand (d_i) and location (x_i, y_i)
    for each customer
    """
    # read in the string to separate lines
    lines = split(input_data, "\n")
    NM = split(lines[1], " ")
    N = parse(Int64, NM[1])
    M = parse(Int64, NM[2])
    
    # create empty array for warehouses + customers
    warehouses = Array{Float64}(undef, 0, 4)
    customers = Array{Float64}(undef, 0, 3)

    # add warehouse details to array
    for line in lines[2:N+1]
        #s_i, cap_i, x_i, y_i = split(line, " ")
        vars = split(line, " ")
        s_i = parse(Float64, vars[1])
        cap_i = parse(Float64, vars[2])
        x_i = parse(Float64, vars[3])
        y_i = parse(Float64, vars[4])
        warehouses = vcat(warehouses, transpose([s_i, cap_i, x_i, y_i]))
    end

    # add customer details to array
    for line in lines[N+2:N+M+1]
        vars = split(line, " ")
        d_i = parse(Float64, vars[1])
        x_i = parse(Float64, vars[2])
        y_i = parse(Float64, vars[3])
        customers = vcat(customers, transpose([d_i, x_i, y_i]))
    end

    # ensure we have the correct length
    @assert size(warehouses)[1] == N
    @assert size(customers)[1] == M
    
    return N, M, warehouses, customers
end


# find the Euclidian distance between each warehouse & customer pair
function get_euclidian_distances(N, M, warehouses, customers)
    """
    Add columns to customers array to show the distance
    between each customer and their specific warehouse.
    """
    for w in 1:N
        x_w, y_w = warehouses[w, 3:end]
        x_C = customers[:, 2]
        y_C = customers[:, 3]
        # calculate distance between warehouses
        d_Cw = ((x_C .- x_w) .^ 2 .+ (y_C .- y_w) .^ 2) .^ (1/2)
        customers = hcat(customers, d_Cw)
    end
    return warehouses, customers
end


function sort_warehouses_by_proximity(N, M, warehouses, customers)
    """
    Sort the warehouses by how close, on average, they are to
    the customers.
    """
    D_avg = sum(customers[:, 4:end], dims=1) ./ M
    warehouse_order = sortperm(D_avg[1, :])

    # reorder the arrays accordingly
    warehouses = warehouses[warehouse_order, :]
    customer_order = vcat([1,2,3], warehouse_order .+ 3)
    customers = customers[:, customer_order]
    
    return warehouses, customers, warehouse_order
end


function initialise_LP_model(N, M, warehouses, customers; solve_time = 1)
    """
    Setup the base LP model for the problem.
    
    Parameters
    ----------
    N: number of warehouses in the problem.
    
    M: number of customers in the problem.
    
    customers: customer statistics (locations and distances
    to warehouses).
    
    warehouses: warehouse statistics (locations and setup
    costs).
    
    Returns
    -------
    lp_model: LP model for solving the facility location
    problem. Contains base constraints for all warehouses
    and customers, where each customer is served by a single
    warehouse, and no warehouse can exceed its capacity.
    """

    # set up the initial LP model
    lp_model = Model(Clp.Optimizer)
    set_time_limit_sec(lp_model, solve_time)
    set_optimizer_attribute(lp_model, MOI.Silent(), true)

    # set up all of our decision vars
    x = @variable(lp_model, [w=1:N], base_name = "x", 
        lower_bound = 0, upper_bound = 1);
    y = @variable(lp_model, [w = 1:N, c = 1:M], base_name = "y", 
        lower_bound = 0, upper_bound = 1);
    
    # number of customers at each x, y location
    n_customers = customers[:, 1]
    
    # ensure that:
    # 1) a customer can only be served by an open warehouse (y_w,c <= x_w)
    # 2) a warehouse cannot exceed its capacity (sum(y_w,c over all C) <= cap_w for w in W)
    for w in 1:N
        # customer only served if warehouse is open
        @constraint(lp_model, y[w, :] .<= x[w], 
            base_name = "warehouse_open_constraint_$w")

        # capacity constraint
        cap_w = warehouses[w, 2]
        @constraint(lp_model, sum(n_customers .* y[w, :]) .<= cap_w, 
            base_name = "warehouse_capacity_constraint_$w")
    end

    # ensure that each customer is fully served by a warehouse
    # (sum(y_w,c = 1 over all W) for c in C)
    for c in 1:M
        # ensure each customer is actually served
        @constraint(lp_model, sum(y[:, c]) .== 1, 
            base_name = "customer_served_constraint_$c")
    end

    # finally, setup our objective function to min. total distance & setup costs
    setup_costs = warehouses[:, 1]
    distance_costs = transpose(customers[:, 4:end])
    @objective(lp_model, Min, sum(setup_costs .* x) + sum(distance_costs .* y))
    
    return lp_model, x, y
end


function add_LP_constraint(lp_model, x, w, open)
    """
    Force a particular warehouse to be either open or shut (0 or 1)
    """
    @constraint(lp_model, x[w] .== open, 
        base_name = "fixed_warehouse_coinstraint_$w")
    return lp_model, x
end


function initialise_MIP_model(N, M, warehouses, customers; solve_time = 1)
    """
    Create a MIP model to assign customers to open warehouses,
    and check for feasible results.
    """
    
    # initialise the MIP model
    mip_model = Model(Cbc.Optimizer)
    set_time_limit_sec(mip_model, solve_time)
    set_optimizer_attribute(mip_model, MOI.Silent(), true)
    
    # set up our decision vars
    y = @variable(mip_model, [w = 1:N, c = 1:M], base_name = "y", 
        binary = true);
    
    # number of customers at each location
    n_customers = customers[:, 1]
    
    # ensure that each warehouse cannot exceed its capacity
    for w in 1:N
        cap_w = warehouses[w, 2]
        @constraint(mip_model, sum(n_customers .* y[w, :]) .<= cap_w, 
            base_name = "warehouse_capacity_constraint_$w")
    end
    
    # ensure that each customer is served by a warehouse
    for c in 1:M
        # ensure each customer is actually served
        @constraint(mip_model, sum(y[:, c]) .== 1, 
            base_name = "customer_served_constraint_$c")
    end
    
    # finally, setup our objective function to min. total distance & setup costs
    setup_costs = warehouses[:, 1]
    distance_costs = transpose(customers[:, 4:4+N-1])
    @objective(mip_model, Min, sum(setup_costs) + sum(distance_costs .* y))
    
    return mip_model, y
end


mutable struct Node
    """
    Nodes to keep track of which facilities are open or shut, and 
    what the estimated value of each node is
    """
    serial_number::String
    estimated_value::Float64
    fully_assessed::Bool
end


function initialise_starting_node(N, M, warehouses, customers)
    
    # build the simple LP model to get an estimate of the 
    # node's best possible value
    LP_model, x, y = initialise_LP_model(N, M, warehouses, customers)
    optimize!(LP_model)
    LP_value = objective_value(LP_model)
    
    # set up our dictionary of nodes
    nodes = Dict()
    serial_number = ""
    nodes[serial_number] = Node(serial_number, LP_value, false)
    return nodes[""], nodes
end    


function create_node(N, M, warehouses, customers, serial_number, nodes)
    """
    Create a node, given a serial number of warehouse choices.
    """
    LP_model, x, y = initialise_LP_model(N, M, warehouses, customers)
    for (w, choice) in enumerate(serial_number)
        open = parse(Int64, choice)
        LP_model, x = add_LP_constraint(LP_model, x, w, open)
    end
    optimize!(LP_model)
    LP_value = objective_value(LP_model)
    node = Node(serial_number, LP_value, false)
    nodes[serial_number] = node
    return node, nodes
end
    
        
function create_child_node(N, M, warehouses, customers, node, nodes; 
        left = true)
    if left == true
        serial_number = node.serial_number * "0"
    else
        serial_number = node.serial_number * "1"
    end
    
    node, nodes = create_node(N, M, warehouses, customers, serial_number, nodes)
    return node, nodes
end


function fill_greedily(N, M, warehouses, customers)
    """
    Pick warehouses until we have 1.25 x the capacity we need.
    """
    node, nodes = initialise_starting_node(N, M, warehouses, customers)
    required_capacity = sum(customers[:, 1]) * 1.25
    
    for i in 1:N
        total_capacity = sum(warehouses[1:i-1, 2])
        if total_capacity < required_capacity
            node, nodes = create_child_node(N, M, warehouses, customers, node, nodes, left=false)
        else
            node, nodes = create_child_node(N, M, warehouses, customers, node, nodes, left=true)
        end
    end
    
    return node, nodes
end
     

function select_available_warehouses(N, warehouses, serial_number)
    available_warehouses = Array{Float64}(undef, 0, 4)
    for w in 1:N
        if serial_number[w] == '1'
            available_warehouses = vcat(available_warehouses, 
                transpose(warehouses[w, :]))
        end
    end
    return available_warehouses
end    


function assess_node(N, M, warehouses, customers, node; solve_time = 1)
    """
    After making all of our warehouse 'choices', assess the result.
    """
    available_warehouses = select_available_warehouses(N, warehouses, node.serial_number)
    N_available, _ = size(available_warehouses)
    mip_model, y = initialise_MIP_model(N_available, M, available_warehouses, customers,
        solve_time = solve_time)
    optimize!(mip_model)
    
    if termination_status(mip_model) == MOI.INFEASIBLE
        obj_value = Inf
        optimal = false
        y_values = nothing
    else
        if termination_status(mip_model) == MOI.OPTIMAL
            optimal = true
        else
            optimal = false
        end
        obj_value = objective_value(mip_model)
        y_values = value.(y)
    end
        
    node.estimated_value = obj_value
    node.fully_assessed = true
    
    return obj_value, y_values, optimal
end


function assess_nodes(N, M, warehouses, customers, nodes)
    """
    Get final values for all terminal nodes.
    """
    for (serial_number, node) in nodes
        if length(node.serial_number) == N
            if node.fully_assessed == false
                assess_node(N, M, warehouses, customers, node)
            end
        end
    end
end


function clip_nodes(nodes; verbose = false)
    """
    Clip out any nodes whose optimistic value is
    worse than the best currently found value.
    """
    
    best_value = Inf
    
    # find the best curent value of nodes that
    # are terminal
    for (serial_number, node) in nodes
        if node.fully_assessed == true
            if node.estimated_value < best_value
                best_value = node.estimated_value
            end
        end
    end
        
    n_clipped = 0
    for (serial_number, node) in nodes
        if node.estimated_value > best_value
            delete!(nodes, node.serial_number)
            n_clipped += 1
        end
    end
    
    if verbose == true
        println("$(n_clipped) nodes clipped.")
    end
end 


function expand_deepest_node(N, M, warehouses, customers, nodes; verbose = false)
    
    deepest_depth = -1
    node_to_expand = nothing
    
    for (serial_number, node) in nodes
        depth = length(node.serial_number)
        if (depth > deepest_depth) & (depth < N)
            node_to_expand = node
        end
    end
    
    # expand the new node (either left or right child)
    if node_to_expand != nothing
        if verbose == true
            println("Expanding node $(node_to_expand.serial_number)")
        end
        if node_to_expand.serial_number * "0" in keys(nodes)
            node, nodes = create_child_node(N, M, warehouses, customers, node_to_expand, nodes; 
                left = false)
        else
            node, nodes = create_child_node(N, M, warehouses, customers, node_to_expand, nodes; 
                left = true)
        end
        # delete the fully expanded node
        delete!(nodes, node_to_expand.serial_number)
    else
        node = node_to_expand
    end
    
    return node, nodes
end


function find_best_result(nodes)
    
    best_value = Inf
    best_node = nothing
    
    if length(nodes) == 1
        optimal = true
        best_node = nodes[collect(keys(nodes))[1]]
    else
        optimal = false
        for (serial_number, node) in nodes
            if node.fully_assessed == true
                if node.estimated_value < best_value
                    best_node = node
                end
            end
        end
    end
    
    return best_node, optimal
end     


function get_result_array(N, node, y_values, warehouse_order)

    warehouse_numbers_selected = [w for (i, w) in enumerate(warehouse_order)
                                  if node.serial_number[i] == '1']

    n_y, n_x = size(y_values)
    result_array = zeros(N, n_x)

    for (i, w) in enumerate(warehouse_numbers_selected)
        result_array[w, :] = y_values[i, :]
    end
                
    return result_array
end
            
            
function format_result(obj_value, optimal, y_values)
    
    result = @sprintf("%.1f", obj_value)
    if optimal == true
        result *= " 1 \n"
    else
        result *= " 0 \n"
    end
                    
    warehouses_selected = findall(x -> x > 0, y_values)
    warehouses_selected = [w[1] for w in warehouses_selected]
    
    for w in warehouses_selected
        result *= "$(w-1) "
    end
                    
    result = result[1:end-1]
    return result
end        
            
            
function solve_facility_location(input_data; timeout = 60)

    # read in the data
    N, M, warehouses, customers = read_input(input_data);
    warehouses, customers = get_euclidian_distances(N, M, warehouses, customers)
    warehouses, customers, warehouse_order = sort_warehouses_by_proximity(N, M, warehouses, customers)

    # create the node structure
    node, nodes = fill_greedily(N, M, warehouses, customers)
    assess_nodes(N, M, warehouses, customers, nodes)
    clip_nodes(nodes)

    # set up timer to avoid timeout limit
    t_limit = Dates.Millisecond(timeout * 1000)
    t0 = Dates.now()
    t1 = Dates.now()

    # optimise nodes using B+B
    while (length(nodes) > 1) & (t1 - t0 < t_limit) & (node != nothing)
        clip_nodes(nodes)
        node, nodes = expand_deepest_node(N, M, warehouses, customers, nodes)
        assess_nodes(N, M, warehouses, customers, nodes)
        clip_nodes(nodes)
        t1 = Dates.now()
    end

    optimal_node, optimal = find_best_result(nodes)
    optimal_obj_value, y_values, optimal_for_node = assess_node(
                    N, M, warehouses, customers, optimal_node, solve_time = 100)
    optimal = (optimal) & (optimal_for_node)
    optimal_y_values = get_result_array(N, optimal_node, y_values, warehouse_order)
    result = format_result(optimal_obj_value, optimal, optimal_y_values)
    
    return result
end

solve_facility_location (generic function with 1 method)

In [82]:
input_data = read("data/problem.txt", String)
result = solve_facility_location(input_data, timeout = 60)

"3436306.6 0 \n7 7 7 7 7 7 0 7 7 7 0 7 7 7 7 7 7 0 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 23 7 7 0 0 7 7 7 7 7 7 7 7 7 7 9 7"

In [84]:
println(result)

3436306.6 0 
7 7 7 7 7 7 0 7 7 7 0 7 7 7 7 7 7 0 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 23 7 7 0 0 7 7 7 7 7 7 7 7 7 7 9 7


In [85]:
length("7 7 7 7 7 7 0 7 7 7 0 7 7 7 7 7 7 0 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 23 7 7 0 0 7 7 7 7 7 7 7 7 7 7 9 7")

100